시나리오2: 1명의 데이터 과학자가 있는 크로스펑셔널 팀. 단일 모델

mlflow setup:
- tracking server: local
- backend server: sqlite 
- artifacts store: local fs

```
mlflow server --backend-store-uri sqlite:///backend.db
```

In [1]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking uri: {mlflow.get_tracking_uri()}")

tracking uri: http://127.0.0.1:5000


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1685507528046, experiment_id='0', last_update_time=1685507528046, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():
    
    X, y = load_iris(return_X_y=True)
    
    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)
    
    lr = LogisticRegression(**params)
    lr.fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    
    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"artifact uri: {mlflow.get_artifact_uri()}")

2023/05/31 13:34:54 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
/Users/ryujaesung/kade/mlops-zoomcamp/venv/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


artifact uri: /Users/ryujaesung/kade/mlops-zoomcamp/week2/running-mlflow-scenarios/artifacts_local/1/f91d0e651e814a9eb3fb17a52e922538/artifacts


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='/Users/ryujaesung/kade/mlops-zoomcamp/week2/running-mlflow-scenarios/artifacts_local/1', creation_time=1685507694913, experiment_id='1', last_update_time=1685507694913, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1685507528046, experiment_id='0', last_update_time=1685507528046, lifecycle_stage='active', name='Default', tags={}>]

interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

In [8]:
client.search_registered_models()

[]

In [12]:
run_id = client.search_runs(experiment_ids="1")[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name="iris-classifier"
)

Successfully registered model 'iris-classifier'.
2023/05/31 13:37:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1685507876286, current_stage='None', description='', last_updated_timestamp=1685507876286, name='iris-classifier', run_id='f91d0e651e814a9eb3fb17a52e922538', run_link='', source='/Users/ryujaesung/kade/mlops-zoomcamp/week2/running-mlflow-scenarios/artifacts_local/1/f91d0e651e814a9eb3fb17a52e922538/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>